# 10-714: Homework 1

This homework will get you started with your implementation of the **needle** (**ne**cessary **e**lements of **d**eep **le**arning) library.  In particular, the goal of this assignment is to build a basic **automatic differentiation** framework, then use this to re-implement the simple two-layer neural network you used for the MNIST digit classification problem in HW0.

In [ ]:
# # Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714
# %cd /content/drive/MyDrive/10714
# !git clone https://github.com/dlsys10714/hw1.git
# %cd /content/drive/MyDrive/10714/hw1

# !pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
# !pip3 install numdifftools

In [1]:
import sys
sys.path.append('./python')
sys.path.append('./apps')
from simple_ml import *

## Introduction to `needle`

For this homework, you will be implementing the basics of automatic differentiation using a `numpy` CPU backend (in later assignments, you will move to your own linear algebra library including GPU code). All code for this assignment will be written in Python.

For the purposes of this assignment, there are two important files in the `needle` library, the `python/needle/autograd.py` file (which defines the basics of the computational graph framework, and also will form the basis of the automatic differentiation framework), and the `python/needle/ops.py`.file (which contains implementations of various operators).

- `Value`: A value computed in a compute graph, i.e., either the output of some operations applied to other `Value` objects, or a constant (leaf) `Value` objects.  We use a generic class here (which we then specialize to e.g. Tensors), in order to allow for other data structures in later version of needle, but for now you will interact with this class mostly through its subclass `Tensor`.
- `Op`: An operator in a compute graph.  Operators need to define their "forward" pass in the `compute()` method (i.e., how to compute the operator on the underlying data of the `Value` objects), as well as their "backward" pass via the `gradient()` method, which defines how to multiply by incoming output gradients.
- `Tensor`: This is a subclass of `Value` that corresponds to an actual tensor output, i.e., a multi-dimensional array within a computation graph.  All of your code for this assignment (and most of the following) will use this subclass of `Value` rather than the generic class above.  We have provided several convenience functions (e.g., operator overloading) that let you operate on tensor using normal Python conventions, but these will not work properly until you implement the corresponding operations.
- `TensorOp`: This is a subclass for `Op` for operators that return Tensors.  All the operations you implement for this assignment will be of this type.


## Question 1: Implementing forward computation [10 pts]


First, you will implement the forward computation for new operators.  To see how this works, consider the `EWiseAdd` operator in the `ops.py` file.

The `compute()` function computes the "forward" pass, i.e., just computes the operation itself.  However, it is important to emphasize the inputs to compute are both `NDArray` objects.  That is, `compute()` computes the forward pass on the _raw data objects_ themselves, not on Tensor objects within the automatic differentiation (manipulation on `Tensor` objects might change the compuete graph).

We will discuss the `gradient()` call in the next section, but it is important to emphasize here that this call is different from forward in that it takes `Tensor` arguments (instead of _raw data objects_).  This means that any call you make within this function _should_ be done via `TensorOp` operations themselves (so that you can take gradients of gradients).

Finally, note that we also define a helper `add()` function, to avoid the need to call `EWiseAdd()(a,b)` (which is a bit cumbersome) to add two `Tensor` objects.  These functions are all written for you, and should be self-explanatory.

For this question, you will need to implement the `compute` call for each of the following classes.  These calls are very straightforward, and should be essentially one line that calls to the relevant numpy function.  Note that because in later homeworks you will use a backend other than numpy, we have imported numpy as `import numpy as array_api`, so that you'll need to call `array_api.add()` etc, if you want to use the typical `np.X()` calls.

- `PowerScalar`
- `EWiseDiv`
- `DivScalar`
- `MatMul`
- `Summation`
- `BroadcastTo`
- `Reshape`
- `Negate`
- `Transpose`

In [2]:
!python3 -m pytest -v -k "forward"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0 -- /home/hujunhao/.conda/envs/dlsys/bin/python3
cachedir: .pytest_cache
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 13 deselected / 8 selected                                

tests/test_autograd_hw.py::test_divide_forward PASSED                    [ 12%]
tests/test_autograd_hw.py::test_divide_scalar_forward PASSED             [ 25%]
tests/test_autograd_hw.py::test_matmul_forward PASSED                    [ 37%]
tests/test_autograd_hw.py::test_summation_forward PASSED                 [ 50%]
tests/test_autograd_hw.py::test_broadcast_to_forward PASSED              [ 62%]
tests/test_autograd_hw.py::test_reshape_forward PASSED                   [ 75%]
tests/test_autograd_hw.py::test_negate_forward PASSED                    [ 87%]
tests/test_autograd_hw.py::test_transpose_forward PASSED                 [100%]

======================= 8 passe

In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "forward"

## Question 2: Implementing backward computation [25 pts]

The easiest way to perform these computations is, again, via taking "fake" partial derivatives (assuming everything is a scalar), and then matching sizes.

### Implementing backward passes

Note that, unlike the forward pass functions, the arguments to the `gradient` function are `needle` objects. It is important to implement the backward passes using only `needle` operations (i.e. those defined in `python/needle/ops.py`), rather than using `numpy` operations on the underlying `numpy` data, so that we can construct the gradients themselves via a computation graph (one exception is for the `ReLU` operation defined below, where you could directly access data within the Tensor without risk because the gradient itself is non-differentiable, but this is a special case).


To complete this question, fill in the `gradient` function of the following classes:

- `EWiseDiv`
- `DivScalar`
- `MatMul`
- `Summation`
- `BroadcastTo`
- `Reshape`
- `Negate`
- `Transpose`

**Hint:** Remember that the size of `out_grad` will always be the size of the _output_ of the operation, whereas the sizes of the `Tensor` objects _returned_ by `gradient()` have to always be the same as the original _inputs_ to the operator.


### Checking backward passes
To reiterate the above, remember that we can check that these backward passes are correct by doing numerical gradient checking as covered in lecture:
\begin{equation}
\delta^T \nabla_\theta f(\theta) = \frac{f(\theta + \epsilon \delta) - f(\theta - \epsilon \delta)}{2 \epsilon} + o(\epsilon^2)
\end{equation}
We provide the function `gradient_check` for doing this numerical checking in `tests/test_autograd.py`.

In [3]:
!python3 -m pytest -l -v -k "backward"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0 -- /home/hujunhao/.conda/envs/dlsys/bin/python3
cachedir: .pytest_cache
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 12 deselected / 9 selected                                

tests/test_autograd_hw.py::test_divide_backward PASSED                   [ 11%]
tests/test_autograd_hw.py::test_divide_scalar_backward PASSED            [ 22%]
tests/test_autograd_hw.py::test_matmul_simple_backward PASSED            [ 33%]
tests/test_autograd_hw.py::test_matmul_batched_backward PASSED           [ 44%]
tests/test_autograd_hw.py::test_reshape_backward PASSED                  [ 55%]
tests/test_autograd_hw.py::test_negate_backward PASSED                   [ 66%]
tests/test_autograd_hw.py::test_transpose_backward PASSED                [ 77%]
tests/test_autograd_hw.py::test_broadcast_to_backward PASSED             [ 88%]
tests/test_autograd_hw.py::test_

In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "backward"

## Question 3: Topological sort [20 pts]

Now your system is capable of performing operations on tensors which builds up a computation graph. Next you will write one of the main utilities needed for automatic differentiation - the [topological sort](https://en.wikipedia.org/wiki/Topological_sorting). This will allow us to traverse through (forward or backward) the computation graph, computing gradients along the way.

Fill out the `find_topo_sort` method and the `topo_sort_dfs` helper method (in `python/needle/autograd.py`) to perform this topological sorting. 

#### Hints: 
- Ensure that you do a post-order depth-first search, otherwise the test cases will fail. 
- The `topo_sort_dfs` method is not required, but we find it useful to use this as a recursive helper function. 
- The "Reverse mode AD by extending computational graph" section of the Lecture 4 slides walks through an example of the proper node ordering. 
- We will be traversing this sorting backwards in later parts of this homework, but the `find_topo_sort` should return the node ordering in the forward direction. 

In [4]:
!python3 -m pytest -k "topo_sort"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 20 deselected / 1 selected                                

tests/test_autograd_hw.py .                                              [100%]

======================= 1 passed, 20 deselected in 0.22s =======================


In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "topo_sort"

## Question 4: Implementing reverse mode differentiation [25 pts]

Once you have correctly implemented the topological sort, you will next leverage it to implement reverse mode automatic differentiation. As a recap from last lecture, we will need to traverse the computational graph in reverse topological order, and construct the new adjoint nodes. For this question, implement the Reverse AD algorithm in the `compute_gradient_of_variables` function in `python/needle/autograd.py`. This will enable use of the `backward` function that computes the gradient and stores the gradient in the `grad` field of each input `Tensor`. With this completed, our reverse mode auto-differentiation engine is functional.

As discussed in lecture the result of reverse mode AD is still a computational graph. We can extend that graph further by composing more operations and run reverse mode AD again on the gradient (the last two tests of this problem). 

In [5]:
!python3 -m pytest -k "compute_gradient"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 20 deselected / 1 selected                                

tests/test_autograd_hw.py .                                              [100%]

======================= 1 passed, 20 deselected in 0.23s =======================


In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "compute_gradient"

## Question 5: Softmax loss [10 pts]

In this question, you will implement the softmax loss as defined in the `softmax_loss()` function in `apps/simple_ml.py`, which we defined in Question 3 of Homework 0, except this time, the softmax loss takes as input a `Tensor` of logits and a `Tensor` of one hot encodings of the true labels. 

Finally, note that the average softmax loss returned should also be a `Tensor`. 

In [6]:
!python3 -m pytest -k "softmax_loss_ndl"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 20 deselected / 1 selected                                

tests/test_autograd_hw.py .                                              [100%]

======================= 1 passed, 20 deselected in 0.59s =======================


In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "softmax_loss_ndl"

## Question 6: SGD for a two-layer neural network [10 pts]

First, you will need to implement the forward and backward passes of the `relu` operator. 
1. Begin by filling out the function `ReLU` operator in `python/needle/ops.py`.
2. Then fill out the `gradient` function of the class `ReLU` in `python/needle/ops.py`.  **Note that in this one case it's acceptable to access the `.realize_cached_data()` call on the output tensor, since the ReLU function is not twice differentiable anyway**.

Then, 

3. Fill out the `nn_epoch` method in the `apps/simple_ml.py` file. 

Note that unlike in Homework 0, the inputs `W1` and `W2` are `Tensors`. Inputs `X` and `y` however are still numpy arrays - you should iterate over mini-batches of the numpy arrays `X` and `y` as you did in Homework 0, and then cast each `X_batch` as a `Tensor`, and one hot encode `y_batch` and cast as a `Tensor`. While last time we derived the backpropagation equations for this two-layer ReLU network directly, this time we will be using our auto-differentiation engine to compute the gradients generically by calling the `.backward()` method of the `Tensor` class. For each mini-batch, after calling `.backward()`, you should compute the updated values for `W1` and `W2` in `numpy`, and then create new `Tensors` for `W1` and `W2` with these `numpy` values. Your solution should return the final `W1` and `W2` `Tensors`. 

In [7]:
!python3 -m pytest -l -k "nn_epoch_ndl"

============================= test session starts ==============================
platform linux -- Python 3.8.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/hujunhao/dlsys/hw1
collected 21 items / 20 deselected / 1 selected                                

tests/test_autograd_hw.py .                                              [100%]

======================= 1 passed, 20 deselected in 2.92s =======================


In [ ]:
!python3 -m mugrade submit 'YOUR_GRADER_KEY_HERE' -k "nn_epoch_ndl"